# Script to prepare data for YOLO training

In [36]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [37]:
# Get csv paths
path_csv_train = "../../Dataset acoustique insectes/CSVs morceaux audio 5s/Audible/train_audible_recording_chunks.csv"
path_csv_test = "../../Dataset acoustique insectes/CSVs morceaux audio 5s/Audible/test_audible_recording_chunks.csv"
path_csv_val = "../../Dataset acoustique insectes/CSVs morceaux audio 5s/Audible/val_audible_recording_chunks.csv"

# Get spectrograms paths
path_spectro_train = "../../Dataset acoustique insectes/Sélection morceaux audio 5s/Audible/train_spectro"
path_spectro_test = "../../Dataset acoustique insectes/Sélection morceaux audio 5s/Audible/test_spectro"
path_spectro_val = "../../Dataset acoustique insectes/Sélection morceaux audio 5s/Audible/val_spectro"

# Using pandas to read the CSV file into a DataFrame
df_train = pd.read_csv(path_csv_train)
df_test = pd.read_csv(path_csv_test)
df_val = pd.read_csv(path_csv_val)

In [38]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(df_train['label'])

def get_data_info(df):
    y = df['label']
    y_category = df['label_category']
    y_kingdom = df['label_kingdom']
    y_class = df['label_class']
    y_order = df['label_order']
    y_family = df['label_family']
    y_subfamily = df['label_subfamily']

    # Convertir les étiquettes en nombres
    y_encoded = label_encoder.transform(y)
    
    # create new empty dataframe
    new_df = pd.DataFrame()

    new_df['t1'] = df["annotation_initial_time"] - df["chunk_initial_time"]
    new_df['t2'] = df["annotation_final_time"] - df["chunk_initial_time"]
    new_df['f1'] = df['min_frequency'].astype(int)
    new_df['f2'] = df['max_frequency'].astype(int)
    new_df['code_unique'] = df['code_unique']
    new_df['chunk_initial_time'] = df['chunk_initial_time']

    # Gestion des valeurs manquantes
    new_df = new_df.dropna()
    
    X = new_df

    return X, y_encoded

info_train, y_train = get_data_info(df_train)
info_test, y_test = get_data_info(df_test)
info_val, y_val = get_data_info(df_val)

In [39]:
info_train

,t1,t2,f1,f2,code_unique,chunk_initial_time
0,0.355751,4.147038,0,375,Anura_XXXXigeXXES_01VII2022_330,0
1,0.000000,5.000000,3248,4453,Anura_XXXXigeXXES_01VII2022_330,0
2,0.000000,5.000000,3248,4453,Anura_XXXXigeXXES_01VII2022_330,5
3,0.346426,1.708443,0,375,Anura_XXXXigeXXES_01VII2022_330,5
4,0.000000,5.000000,3248,4453,Anura_XXXXigeXXES_01VII2022_330,10
...,...,...,...,...,...,...
10784,2.296621,3.300916,5007,16160,Aves_KrDe_AwSw00PL_16VII2014_4613,175
10785,4.691478,5.000000,4750,16553,Aves_KrDe_AwSw00PL_16VII2014_4613,175
10786,0.000000,0.674299,5416,17367,Aves_KrDe_AwSw00PL_16VII2014_4613,175
10787,3.399642,3.863163,5007,15776,Aves_KrDe_AwSw00PL_16VII2014_4613,180
